<a href="https://colab.research.google.com/github/FMGS666/NLPProject/blob/trail_model/Copy_of_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = stopwords.words('english')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!pip install transformers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.6 MB/s eta 0:00:00


In [ ]:
path = '/content/traindata.csv'
pathdev = '/content/devdata.csv'
df = pd.read_csv(path,sep='\t', header=None)
df_dev = pd.read_csv(pathdev,sep='\t', header=None)

In [ ]:
df.columns = ['polarity', 'aspect_category', 'target_term', 'character_offsets', 'sentence']
df_dev.columns = ['polarity', 'aspect_category', 'target_term', 'character_offsets', 'sentence']

In [ ]:
df.head()

,polarity,aspect_category,target_term,character_offsets,sentence
0,positive,AMBIENCE#GENERAL,seating,18:25,short and sweet – seating is great:it's romant...
1,positive,AMBIENCE#GENERAL,trattoria,25:34,This quaint and romantic trattoria is at the t...
2,positive,FOOD#QUALITY,food,98:102,The have over 100 different beers to offer thi...
3,negative,SERVICE#GENERAL,STAFF,5:10,THIS STAFF SHOULD BE FIRED.
4,positive,FOOD#STYLE_OPTIONS,menu,4:8,"The menu looked great, and the waiter was very..."


In [ ]:
df.polarity.value_counts()

positive    1055
negative     390
neutral       58
Name: polarity, dtype: int64

In [ ]:
df.sentence.duplicated().sum()

440

In [ ]:
#label encoding the polarity
lb = preprocessing.LabelEncoder()
df['polarity']=lb.fit_transform(df['polarity'])
df_dev['polarity']=lb.fit_transform(df_dev['polarity'])


In [ ]:
def add_sep(text,offset):
    sep = ' [SEP] '
    split = offset.split(':')
    sentence_left = text[:int(split[0])]
    target = text[int(split[0]):int(split[1])]
    sentence_right = text[int(split[1]):]
    concat = sentence_left+sep+target+sep+sentence_right
    return concat

In [ ]:
df['sentence_sep'] = df.apply(lambda x: add_sep(x['sentence'],x['character_offsets']),axis = 1)

In [ ]:
df_dev['sentence_sep'] = df_dev.apply(lambda x: add_sep(x['sentence'],x['character_offsets']),axis = 1)

## BERT

In [ ]:
df['target_term'] = df['target_term'].apply(lambda x: x.lower())
df['aspect_category'] = df['aspect_category'].apply(preprocess_category)

In [ ]:
df_dev['target_term'] = df_dev['target_term'].apply(lambda x: x.lower())
df_dev['aspect_category'] = df_dev['aspect_category'].apply(preprocess_category)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

# initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.to(device)

# tokenize the sentences in the training and validation sets
train_tokens = tokenizer.batch_encode_plus(train_df.apply(lambda x: f"{x['target_term']} {x['sentence']}", axis=1).tolist(), 
                                           max_length=128, 
                                           padding='max_length', 
                                           truncation=True, 
                                           return_tensors='pt')
val_tokens = tokenizer.batch_encode_plus(val_df.apply(lambda x: f"{x['target_term']} {x['sentence']}", axis=1).tolist(), 
                                         max_length=128, 
                                         padding='max_length', 
                                         truncation=True, 
                                         return_tensors='pt')

# create PyTorch DataLoader objects for the training and validation sets
train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], torch.tensor(train_df['polarity'].tolist()))
val_dataset = TensorDataset(val_tokens['input_ids'], val_tokens['attention_mask'], torch.tensor(val_df['polarity'].tolist()))
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=16)

# set the optimizer and learning rate scheduler
epochs=15
# calculate class weights
#class_weights = compute_class_weight('balanced', classes=np.unique(train_df['polarity']), y=train_df['polarity'])
class_weights = np.array([3.,40.,1.])

class_weights = torch.FloatTensor(class_weights).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*epochs)

# define the training loop
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            val_loss += loss.item() * input_ids.size(0)
            val_preds += torch.argmax(logits, axis=1).tolist()
            val_labels += labels.tolist()
    val_loss /= len(val_loader.dataset)
    val_accuracy = accuracy_score(val_labels, val_preds)
    val_precision = precision_score(val_labels, val_preds, average='weighted', zero_division=1)
    val_recall = recall_score(val_labels, val_preds, average='weighted')
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    report = classification_report(val_labels, val_preds, labels=[0, 1, 2], target_names=['Negative', 'Neutral', 'Positive'])
    print(f'Epoch {epoch+1} - Val Loss: {val_loss:.4f} - Val Accuracy: {val_accuracy:.4f} - Val Precision: {val_precision:.4f} - Val Recall: {val_recall:.4f} - Val F1: {val_f1:4f}')
    print(report)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1 - Val Loss: 0.5503 - Val Accuracy: 0.7938 - Val Precision: 0.7939 - Val Recall: 0.7938 - Val F1: 0.752702
              precision    recall  f1-score   support

    Negative       0.74      0.35      0.48       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.80      0.98      0.88       329

    accuracy                           0.79       451
   macro avg       0.51      0.44      0.45       451
weighted avg       0.76      0.79      0.75       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2 - Val Loss: 0.4203 - Val Accuracy: 0.8293 - Val Precision: 0.8519 - Val Recall: 0.8293 - Val F1: 0.818622
              precision    recall  f1-score   support

    Negative       0.63      0.82      0.71       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.91      0.88      0.89       329

    accuracy                           0.83       451
   macro avg       0.52      0.56      0.54       451
weighted avg       0.81      0.83      0.82       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch 6 - Val Loss: 0.5493 - Val Accuracy: 0.8537 - Val Precision: 0.8569 - Val Recall: 0.8537 - Val F1: 0.835741
              precision    recall  f1-score   support

    Negative       0.75      0.72      0.74       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.88      0.94      0.91       329

    accuracy                           0.85       451
   macro avg       0.55      0.55      0.55       451
weighted avg       0.82      0.85      0.84       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 7 - Val Loss: 0.5327 - Val Accuracy: 0.8514 - Val Precision: 0.8570 - Val Recall: 0.8514 - Val F1: 0.835065
              precision    recall  f1-score   support

    Negative       0.72      0.75      0.74       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.89      0.93      0.91       329

    accuracy                           0.85       451
   macro avg       0.54      0.56      0.55       451
weighted avg       0.82      0.85      0.84       451

Epoch 8 - Val Loss: 0.5357 - Val Accuracy: 0.8581 - Val Precision: 0.8652 - Val Recall: 0.8581 - Val F1: 0.845497
              precision    recall  f1-score   support

    Negative       0.72      0.78      0.75       105
     Neutral       1.00      0.06      0.11        17
    Positive       0.90      0.92      0.91       329

    accuracy                           0.86       451
   macro avg       0.87      0.59      0.59       451
weighted avg       0.87      0.86      0.85       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 9 - Val Loss: 0.5581 - Val Accuracy: 0.8581 - Val Precision: 0.8657 - Val Recall: 0.8581 - Val F1: 0.842633
              precision    recall  f1-score   support

    Negative       0.72      0.79      0.75       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.90      0.92      0.91       329

    accuracy                           0.86       451
   macro avg       0.54      0.57      0.56       451
weighted avg       0.83      0.86      0.84       451

Epoch 10 - Val Loss: 0.5605 - Val Accuracy: 0.8559 - Val Precision: 0.8465 - Val Recall: 0.8559 - Val F1: 0.844211
              precision    recall  f1-score   support

    Negative       0.72      0.79      0.75       105
     Neutral       0.50      0.06      0.11        17
    Positive       0.90      0.92      0.91       329

    accuracy                           0.86       451
   macro avg       0.71      0.59      0.59       451
weighted avg       0.85      0.86      0.84       451

Epoch 11 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

# initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.to(device)

# tokenize the sentences in the training and validation sets
train_tokens = tokenizer.batch_encode_plus(train_df.apply(lambda x: f"{x['target_term']} {x['sentence_sep']}", axis=1).tolist(), 
                                           max_length=256, 
                                           padding='max_length', 
                                           truncation=True, 
                                           return_tensors='pt')
val_tokens = tokenizer.batch_encode_plus(val_df.apply(lambda x: f"{x['target_term']} {x['sentence_sep']}", axis=1).tolist(), 
                                         max_length=128, 
                                         padding='max_length', 
                                         truncation=True, 
                                         return_tensors='pt')

# create PyTorch DataLoader objects for the training and validation sets
train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], torch.tensor(train_df['polarity'].tolist()))
val_dataset = TensorDataset(val_tokens['input_ids'], val_tokens['attention_mask'], torch.tensor(val_df['polarity'].tolist()))
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=16)

# set the optimizer and learning rate scheduler
epochs=15
# calculate class weights
class_weights = np.array([3.,20.,1.])
class_weights = torch.FloatTensor(class_weights).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*epochs)

# define the training loop
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            val_loss += loss.item() * input_ids.size(0)
            val_preds += torch.argmax(logits, axis=1).tolist()
            val_labels += labels.tolist()
    val_loss /= len(val_loader.dataset)
    val_accuracy = accuracy_score(val_labels, val_preds)
    val_precision = precision_score(val_labels, val_preds, average='weighted', zero_division=1)
    val_recall = recall_score(val_labels, val_preds, average='weighted')
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    report = classification_report(val_labels, val_preds, labels=[0, 1, 2], target_names=['Negative', 'Neutral', 'Positive'])
    print(f'Epoch {epoch+1} - Val Loss: {val_loss:.4f} - Val Accuracy: {val_accuracy:.4f} - Val Precision: {val_precision:.4f} - Val Recall: {val_recall:.4f} - Val F1: {val_f1:4f}')
    print(report)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1 - Val Loss: 0.6175 - Val Accuracy: 0.7916 - Val Precision: 0.7862 - Val Recall: 0.7916 - Val F1: 0.761469
              precision    recall  f1-score   support

    Negative       0.66      0.44      0.53       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.82      0.95      0.88       329

    accuracy                           0.79       451
   macro avg       0.49      0.46      0.47       451
weighted avg       0.75      0.79      0.76       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2 - Val Loss: 0.4334 - Val Accuracy: 0.8226 - Val Precision: 0.8593 - Val Recall: 0.8226 - Val F1: 0.814863
              precision    recall  f1-score   support

    Negative       0.61      0.88      0.72       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.93      0.85      0.89       329

    accuracy                           0.82       451
   macro avg       0.51      0.57      0.53       451
weighted avg       0.82      0.82      0.81       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 3 - Val Loss: 0.4906 - Val Accuracy: 0.8271 - Val Precision: 0.8669 - Val Recall: 0.8271 - Val F1: 0.819477
              precision    recall  f1-score   support

    Negative       0.61      0.90      0.73       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.94      0.84      0.89       329

    accuracy                           0.83       451
   macro avg       0.52      0.58      0.54       451
weighted avg       0.83      0.83      0.82       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 4 - Val Loss: 0.5209 - Val Accuracy: 0.8559 - Val Precision: 0.8582 - Val Recall: 0.8559 - Val F1: 0.836390
              precision    recall  f1-score   support

    Negative       0.78      0.70      0.74       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.87      0.95      0.91       329

    accuracy                           0.86       451
   macro avg       0.55      0.55      0.55       451
weighted avg       0.82      0.86      0.84       451



KeyboardInterrupt: ignored

In [ ]:
dev_tokens = tokenizer.batch_encode_plus(df_dev.apply(lambda x: f"{x['sentence_sep']}", axis=1).tolist(), 
                                         max_length=128, 
                                         padding='max_length', 
                                         truncation=True, 
                                         return_tensors='pt')

# create PyTorch DataLoader objects for the training and validation sets

dev_dataset = TensorDataset(dev_tokens['input_ids'], dev_tokens['attention_mask'], torch.tensor(df_dev['polarity'].tolist()))
dev_loader = DataLoader(dev_dataset, batch_size=16)

model.eval()
dev_loss = 0
dev_preds = []
dev_labels = []
with torch.no_grad():
    for batch in dev_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            dev_loss += loss.item() * input_ids.size(0)
            dev_preds += torch.argmax(logits, axis=1).tolist()
            dev_labels += labels.tolist()
dev_loss /= len(dev_loader.dataset)
dev_accuracy = accuracy_score(dev_labels, dev_preds)
dev_precision = precision_score(dev_labels, dev_preds, average='weighted', zero_division=1)
dev_recall = recall_score(dev_labels, dev_preds, average='weighted')
dev_f1 = f1_score(dev_labels, dev_preds, average='weighted')
report = classification_report(dev_labels, dev_preds, labels=[0, 1, 2], target_names=['Negative', 'Neutral', 'Positive'])
print(f'Dev Loss: {dev_loss:.4f} - Dev Accuracy: {dev_accuracy:.4f} - Dev Precision: {dev_precision:.4f} - Dev Recall: {dev_recall:.4f} - Val F1: {dev_f1:4f}')
print(report)

Dev Loss: 0.6772 - Dev Accuracy: 0.8378 - Dev Precision: 0.8436 - Dev Recall: 0.8378 - Val F1: 0.839968
              precision    recall  f1-score   support

    Negative       0.70      0.78      0.74        98
     Neutral       0.36      0.36      0.36        14
    Positive       0.92      0.89      0.90       264

    accuracy                           0.84       376
   macro avg       0.66      0.67      0.67       376
weighted avg       0.84      0.84      0.84       376



## Roberta

In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.utils.class_weight import compute_class_weight

# set the device to run the model on (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

# initialize the RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
model.to(device)

# tokenize the sentences in the training and validation sets
train_tokens = tokenizer.batch_encode_plus(train_df['sentence_sep'].tolist(), 
                                           max_length=256, 
                                           padding='max_length', 
                                           truncation=True, 
                                           return_tensors='pt')
val_tokens = tokenizer.batch_encode_plus(val_df['sentence_sep'].tolist(), 
                                         max_length=256, 
                                         padding='max_length', 
                                         truncation=True, 
                                         return_tensors='pt')

# create PyTorch DataLoader objects for the training and validation sets
train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], torch.tensor(train_df['polarity'].tolist()))
val_dataset = TensorDataset(val_tokens['input_ids'], val_tokens['attention_mask'], torch.tensor(val_df['polarity'].tolist()))
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=32)
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)

# set the optimizer and learning rate scheduler
epochs = 15
# define the class weights as a tensor
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['polarity']), y=train_df['polarity'])
class_weights = torch.FloatTensor(class_weights).to(device)
#class_weights = torch.tensor([3., 20., 1.])
# define the loss function with the weighted cross entropy loss
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*epochs)

# define the training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
    train_loss /= len(train_loader)
    
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            val_loss += loss.item() * input_ids.size(0)
            val_preds += torch.argmax(logits, axis=1).tolist()
            val_labels += labels.tolist()
    val_loss /= len(val_loader.dataset)
    val_accuracy = accuracy_score(val_labels, val_preds)
    val_precision = precision_score(val_labels, val_preds, average='weighted', zero_division=1)
    val_recall = recall_score(val_labels, val_preds, average='weighted')
    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    report = classification_report(val_labels, val_preds, labels=[0, 1, 2], target_names=['Negative', 'Neutral', 'Positive'])
    print(f'Epoch {epoch+1} - Val Loss: {val_loss:.4f} - Val Accuracy: {val_accuracy:.4f} - Val Precision: {val_precision:.4f} - Val Recall: {val_recall:.4f} - Val F1: {val_f1:4f}')
    print(report)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch 1 - Val Loss: 0.6399 - Val Accuracy: 0.7295 - Val Precision: 0.8027 - Val Recall: 0.7295 - Val F1: 0.615390
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.73      1.00      0.84       329

    accuracy                           0.73       451
   macro avg       0.24      0.33      0.28       451
weighted avg       0.53      0.73      0.62       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2 - Val Loss: 0.4109 - Val Accuracy: 0.8758 - Val Precision: 0.8961 - Val Recall: 0.8758 - Val F1: 0.863495
              precision    recall  f1-score   support

    Negative       0.69      0.90      0.79       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.96      0.91      0.93       329

    accuracy                           0.88       451
   macro avg       0.55      0.61      0.57       451
weighted avg       0.86      0.88      0.86       451



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 3 - Val Loss: 0.3475 - Val Accuracy: 0.8714 - Val Precision: 0.8900 - Val Recall: 0.8714 - Val F1: 0.858706
              precision    recall  f1-score   support

    Negative       0.70      0.90      0.78       105
     Neutral       0.00      0.00      0.00        17
    Positive       0.95      0.91      0.93       329

    accuracy                           0.87       451
   macro avg       0.55      0.60      0.57       451
weighted avg       0.85      0.87      0.86       451

Epoch 4 - Val Loss: 0.3035 - Val Accuracy: 0.8980 - Val Precision: 0.8915 - Val Recall: 0.8980 - Val F1: 0.887264
              precision    recall  f1-score   support

    Negative       0.80      0.85      0.82       105
     Neutral       0.67      0.12      0.20        17
    Positive       0.93      0.95      0.94       329

    accuracy                           0.90       451
   macro avg       0.80      0.64      0.66       451
weighted avg       0.89      0.90      0.89       451

Epoch 5 - 

In [ ]:
dev_tokens = tokenizer.batch_encode_plus(df_dev.apply(lambda x: f"{x['sentence_sep']}", axis=1).tolist(), 
                                         max_length=128, 
                                         padding='max_length', 
                                         truncation=True, 
                                         return_tensors='pt')

# create PyTorch DataLoader objects for the training and validation sets

dev_dataset = TensorDataset(dev_tokens['input_ids'], dev_tokens['attention_mask'], torch.tensor(df_dev['polarity'].tolist()))
dev_loader = DataLoader(dev_dataset, sampler=SequentialSampler(dev_dataset), batch_size=16)
model.eval()
dev_loss = 0
dev_preds = []
dev_labels = []
with torch.no_grad():
    for batch in dev_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            dev_loss += loss.item() * input_ids.size(0)
            dev_preds += torch.argmax(logits, axis=1).tolist()
            dev_labels += labels.tolist()
dev_loss /= len(dev_loader.dataset)
dev_accuracy = accuracy_score(dev_labels, dev_preds)
dev_precision = precision_score(dev_labels, dev_preds, average='weighted', zero_division=1)
dev_recall = recall_score(dev_labels, dev_preds, average='weighted')
dev_f1 = f1_score(dev_labels, dev_preds, average='weighted')
report = classification_report(dev_labels, dev_preds, labels=[0, 1, 2], target_names=['Negative', 'Neutral', 'Positive'])
print(f'Dev Loss: {dev_loss:.4f} - Dev Accuracy: {dev_accuracy:.4f} - Dev Precision: {dev_precision:.4f} - Dev Recall: {dev_recall:.4f} - Val F1: {dev_f1:4f}')
print(report)

Dev Loss: 0.5083 - Dev Accuracy: 0.8777 - Dev Precision: 0.8750 - Dev Recall: 0.8777 - Val F1: 0.876048
              precision    recall  f1-score   support

    Negative       0.79      0.82      0.80        98
     Neutral       0.45      0.36      0.40        14
    Positive       0.93      0.93      0.93       264

    accuracy                           0.88       376
   macro avg       0.72      0.70      0.71       376
weighted avg       0.87      0.88      0.88       376



In [ ]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             